<img src="../inputs/section_header.png">

# Python Visualisation Tutorial

Hello there, and welcome to our visualisation workshop!

This tutorial is entirely dedicated to offer useful tools to get started with visualisations in Python, with examples on neuro-imaging data. 

The data that we will explore today is part of an [open source repository](https://openneuro.org/datasets/ds003542) on adult language learners. We express our sincere gratitude to Kshipra Gurunandan, Manuel Carreiras and Pedro M. Paz-Alonso for making this data open source, and thus allowing us to make this tutorial.

***

## Wait, what ...? A Jupyter notebook?

Never worked with jupyter notebooks before? Don't worry! We'll guide you through it step by step.

A **jupyter notebook** 
- is a convenient way to combine code with neat documentation, allowing others to easily follow a project that uses code
- works with so-called "blocks", which can either be a "code block" or a "markdown block"
    - *Markdown block*: You are reading one right now! It is a block that understands the markdown language, an easy language to create text, similar to HTML and LaTeX.
    - *Code block*: A code block in a jupyter notebook understands the Python language by default. If applicable, output is printed below the code block, such as images and text. You can also change the kernel of code blocks; in this way you can use other programming languages in the code blocks! Read more about this [here](https://jupyter4edu.github.io/jupyter-edu-book/jupyter.html).
- Allows to "hide" blocks, which will be visible with 3 dots (...). Solutions are hidden this way, but first try to find the solution yourself in the empty block above!
- has some nice shortcuts:

|Operation|Shortcut|
|---|---|
|Run a cell|shift + enter / ctrl + enter|
|Add cell above | "a" (first click left of the cell)|
|Add cell below | "b" (first click left of the cell)|
|Copy cell: | "c" (first click left of the cell)|
|Cut cell: | "x" (first click left of the cell)|
|Paste cell | "v" (first click left of the cell)|
|Delete cell | "dd" (first click left of the cell)|
|Comment out code | select code and press ctrl + / or cmd + /|
|Uncomment code| select code and press ctrl + / or cmd + /|
|Help and documentation| type `?` or `help()` in a code cell|

Last note: If you want to install additional python packages, you can call upon the terminal with an exclamation mark (!) before a command. So, installing a new package will be the following structure inside a code cell: `!pip install my-package`

<br />

***

<br />

## Let's get started!
### Table of contents
[0. Preparations](#section_0)

[1. Visualising neuro-imaging data](#section_1)
    
- [1.1 What is a NIfTI (.nii)?](#section_1.1)
- [1.2 Anatomical MRI - T1w](#section_1.2)
- [1.3 Functional MRI (fMRI)](#section_1.3)

[2. Visualisations on dataframes](#section_2)
- [2.1 Dataframe exploration](#section_2.1)
- [2.2 Basic visualisation with Matplotlib](#section_2.2)
- [2.3 Seaborn](#section_2.3)
- [2.4 Using functions to manipulate a plot](#section_2.4)
- [2.5 Visualisation options for data exploration](#section_2.5)

[3. Integration exercise](#section_3)

[4. Additional plots to consider](#section_4)

[Funny Note :)](#section_funny_note)

[Acknowledgements and References](#section_ack_ref)

<br />

***

<div id="section_0"></div>

# 0. Preparations

Import some packages and functions. The self-written functions can be consulted in the subdirectory python/functions.py

In [23]:
# Required packages for this notebook
import os
from bids import BIDSLayout      # BIDS: For neat neuro-imaging data organisation --> https://bids.neuroimaging.io/
import pandas as pd              # Data handling
import numpy as np               # Linear algebra package
import matplotlib.pyplot as plt  # Visualisation package
import seaborn as sns            # Visualisation package
import json                      # To import json files
import nibabel as nib            # Neuro-imaging library to work with NI data
import nilearn                   # Neuro-imaging library to work with NI data

# Self-made functions
from python.functions import show_slice, track_voxel, simulate_volume

Some notebook settings

In [24]:
# Set matplotlib inline to view images below the cell in this notebook
%matplotlib inline 

# To prettify python code (python syntax is already pretty of course ;) ) --> https://pypi.org/project/nb-black/
%load_ext nb_black 

<IPython.core.display.Javascript object>

Preparations for data imports

In [51]:
data_folder = "../inputs/ds003542"
layout = BIDSLayout(data_folder)

# Description of the project
with open(os.path.join(data_folder, "README")) as txt_file:
    README_sections = txt_file.read().split(f'{"-"*34}')
    project_description = README_sections[0]
    references = README_sections[1]
    contact = README_sections[3]

<IPython.core.display.Javascript object>

***

# The project where the data originates:

Below, we will first print the README file that contains the explanation about the objective of this study.

In [115]:
print("Description from the project's README:")
print('-'*30)
print(project_description)

<IPython.core.display.Javascript object>

***

<img src="../inputs/section_header.png">

<div id="section_1"></div>

# 1. Visualising neuro-imaging data

The data was downloaded from OpenNeuro, [project ds003542](https://openneuro.org/datasets/ds003542/versions/1.0.0), which originates from a study on **language learners**. For the neuro-imaging data part, we only extracted data from subject 1. Hence, the data is present in the "inputs/ds00352" folder, subdirectory "sub-01":
- **Anatomical MRI:** "anat" subfolder
- **Functional MRI:** "func" subfolder

Let's define the paths. We use the [BIDS data organisation](https://bids.neuroimaging.io/) to get the BIDS dataset from the data folder. In short, BIDS aims to uniformise neuro-imaging (meta)data organisation and sharing; essentially speaking the same language when it comes to neuro-imaging data! In Python, we use the package called `pybids` (check [this link](https://pypi.org/project/pybids/)) to facilitate working with BIDS. 

In [33]:
# Index [0] at the end: to get the string from the 1-element vector.
t1w_path = layout.get(
    subject="01", suffix="T1w", extension=".nii.gz", return_type="filename"
)[0]
bold_path = layout.get(
    subject="01",
    task="compL1",
    run="1",
    suffix="bold",
    extension=".nii.gz",
    return_type="filename",
)[0]

<IPython.core.display.Javascript object>

<div id="section_1.1"></div>

## 1.1 What is a NIfTI (.nii)?

For a thorough explanation on the origin and objective of NIfTI, we refer to [this webpage](https://NIfTI.nimh.nih.gov/). In short however, the NIfTI file format has been proposed to 

<div id="section_1.2"></div>

## 1.2 Anatomical MRI - T1w

Click [here](https://nipy.org/nibabel/coordinate_systems.html) for the source article in nibabel.\
Note that this anatomical T1w NIfTI is a 3D image, with each dimension being a spatial dimension (x, y and z), corresponding to a body axis (medio-lateral, antero-posterior and cranio-caudal).

### Let's first search for the NIfTI file in our directory! 

In [34]:
t1w_file = nib.load(t1w_path)  # Upload the NIfTI file with nibabel. The path indicates the NIfTI's location.
t1w_image = t1w_file.get_fdata()  # Get the 3D numpy array that contains grey values of the voxels

<IPython.core.display.Javascript object>

### What are the dimensions of the numpy array?

In [38]:
t1w_shape = ...
print(f"Shape of the 3D numpy array: {...}")
print(f"- # Medio-lateral voxels: {...}")
print(f"- # Antero-posterior voxels: {...}")
print(f"- # Cranio-caudal voxels: {...}")

<IPython.core.display.Javascript object>

[<ins> Solution: click here</ins>](#solution_dim_anat)

### Let's show a single slice of the 3D numpy array.
To do this, we can index the object, for which step 2 will help us!\
We want to extract axial slice 150. Call this variable `t1w_slice`.

Hint: indexing of the `t1w_image` object will be as follows: `t1w_image[ML_direction, AP_direction, CC_direction]`, with each element consisting of either a single value, or a range (e.g. `5:9`). To select everyting along an axis, just pass `:`. ML = Medio-lateral, AP = Antero-posterior, CC = Cranio-caudal.

[<ins> Solution: click here</ins>](#solution_show_anat_slice)

Let's show it!

In [44]:
fig, ax = plt.subplots()
ax.imshow(t1w_slice.T, cmap="gray", origin="lower")

<IPython.core.display.Javascript object>

**A bit more explanation**

The code block that you just ran will partly be covered in [section 2](#section_2). In short:
- The `imshow` function on the ax object will plot the `t1w_slice` on an `ax` object. 
- The `.T` means that you transpose the 2D numpy array, essentially "flipping" the grey values in the 2D matrix. Try removing it and rerunning the cell, you'll see what it does! 
- `cmap` is the color map. Depending on the value, the shade will range from white to black; a good way to visualise an anatomical image!

We have also created a function (`show_slice`) for you to easily investigate your T1w image. You can consult the source code in the `functions.py` file. 

To use it, just run `show_slice(image = t1w_image, orientation = 'axial', slice_nr = 150)` in the next cell. Play around with the `orientation` and `slice_nr` parameters at will!

**REMARK: Defacing**\
Run the following code block in a new cell: `show_slice(image = t1w_image, orientation = 'sagittal', slice_nr = 100)`.\
In this slice, you can nicely see that this NIfTI image has been "defaced", to ensure anonimity. The face part is entirely black. Inside the 3D numpy array, all values that relate to the face were set to 0 (black) here!

<div id="section_1.3"></div>

## 1.3 Functional MRI (fMRI)

First things first: a functional MRI image has a fourth dimension, time!\
Hence, a NIfTI file is a 4D numpy array, with the first 3 dimensions being spatial (x: medio-lateral, y: antero-posterior and z: craniocaudal), and the 4th being time!

EXTRA: For the most fancy experience, you can replace the `color_map` parameter inside the `show_functional_slice` and `track_voxel` functions with one of the options listed in [this link](https://matplotlib.org/stable/tutorials/colors/colormaps.html). This will change the colors of your image, check it out! :) 

### Let's first search for the NIfTI file in our directory! 

Try to upload the files yourself! Check section 1.2 for this. Call your variables `bold_file` and `bold_image` to work with the rest of the notebook.

[<ins> Solution: click here </ins>](#solution_upload_bold_file)

### What are the dimensions of the numpy array?

Try to do this using the similar code block of section 1.2. Remember that the data has a fourth dimension!

[<ins> Solution: click here </ins>](#solution_dim_array_func)

### Let's show a single slice again!
Get inspiration from section 1.2.3 and 1.3.2 to answer this one. We want to visualize axial slice 10.

[<ins> Solution: click here </ins>](#solution_show_bold_image)

Question: When looking at the spatial dimensions and compare it to the T1w image, what do you notice?

[<ins> Solution: click here </ins>](#solution_show_bold_image_question)

We have also created a function (`show_slice`) for you to easily investigate your fMRI image. You can consult the source code in the `functions.py` file. 

To use it, just run `show_slice(image = bold_image, orientation = 'axial', slice_nr = 10, timepoint = 0)` in the next cell. Play around with the `orientation`, `slice_nr` and `timepoint` parameters at will!

### Additional neuro-imaging tools

The "Nilearn" package offers quite some nice tools to visualise your data! Before getting started, let's import nilearn: `import nilearn`. Let's go over some nice options within the "plotting" section:
- `nilearn.plotting.plot_anat(path)`: plot your anatomical image. Only a path to your NIfTI file is required (as can be read [here](http://nilearn.github.io/manipulating_images/input_output.html))! You can read more about the `plot_anat()` function [here](https://nilearn.github.io/modules/generated/nilearn.plotting.plot_anat.html#nilearn.plotting.plot_anat).
- `nilearn.plotting.plot_carpet(path)`: A carpet plot allows you to see the BOLD values of your voxels over time. Later in this notebook, we will show how you can plot the value of a single voxel over time, but this immediately gives you the full picture! Moreover, it takes the time between slices into account, whereas the 4th dimension (time) of our 4D numpy array makes abstraction of this. You can read more about `plot_carpet()` [here](https://nilearn.github.io/auto_examples/01_plotting/plot_carpet.html).

***

<img src="../inputs/section_header.png">

<div id="section_2"></div>

# 2. Visualisation on a dataframe

First some preparations. 

1. We will first upload the following: 
    - data: `participants.tsv`
    - description of the data: `participants.json`

In [7]:
# Data. This is a .tsv (tab-separated) file
table_path = os.path.join(data_folder, "participants.tsv")
df = pd.read_csv(table_path, sep="\t") # Separator "\t" since it is a .tsv file
df['sex'] = df['sex'].apply(lambda x: x.replace(' ', ''))  # Remove the space after M and F

# Data description. This is within a json file, that needs a special way of reading:
with open(os.path.join(data_folder, "participants.json")) as json_file:
    df_description = json.load(json_file)
    df_description = pd.DataFrame(df_description)
    df_description = df_description.drop("TermURL", axis=0)

2. Additionally, we are going to add some synthetic data. We will calculate whole brain and gray matter volume, for which we will use the following regression equations (calculated on segmented open source healthy control data):
$$WholeBrain = 1696.06 - 3.34Age$$
$$GrayMatter = 1041.07 - 2.84Age$$
We will also add some Gaussian noise to both data vectors. We will use the pandas `.apply()` function, applied to a pandas Series object. The `simulate_volume` function will be applied, passing 2 extra arguments, being the `intercept` and the `slope` of the regression equation. The first argument, `age`, will be provided by the Series object.

In [8]:
df['whole_brain'] = df['age'].apply(simulate_volume, args = (1696.06, -3.34))
df['gray_matter'] = df['age'].apply(simulate_volume, args = (1041.07, -2.84))

<div id="section_2.1"></div>

## 2.1 Explore the dataframes quantitatively

Get a sneak preview of the data `df.head()` and the full data description `df_description`

Get a quick insight in the distribution of the data. For this, you can use e.g. `df.describe()` and `df.info()`

<div id="section_2.2"></div>

## 2.2 Basic visualisation with matplotlib

Let's go through the basics of matplotlib first in order to visualize how your data is distributed. We will use matplotlib.pyplot, which we will import under the name `plt`. Thus, make sure to run this command: `import matplotlib.pyplot as plt`. A general way to start plotting:
1. Initiate a figure instance: `fig = plt.Figure()`
2. Add the desired plot. In this case, we will add a histogram: `plt.hist(x)`, x being the data vector to show (e.g. a column).
3. Tidy up your figure. Quick note: you can change the font sizes with an extra argument (`fontsize = ...`) to all of the functions below. Some quick things we can change (examples: x axis, same applies to y):
    - Labels: `plt.xlabel('label')`
    - Ticklabels sizes: `plt.xticks(fontsize = ...)`
    - Title: `plt.title('title')`
4. End with `plt.show()` to show the figure

Now it's up to you! Plot the distribution of the `age` column with a histogram.

[<ins> Solution: click here </ins>](#solution_first_plt_plot)

Now, we will plot the histogram per sex, one for male, one for female, side by side. For this, instead of `fig = plt.Figure()`, we will make use of `fig, ax = plt.subplots()`. By default, the effect is the same as the former approach, however, this option will allow you to plot multiple plots.
- Initiate fig and ax instance `fig, ax = plt.subplots(nrows = ..., ncols = ..., figsize = ())`
- Use the ax instance to manipulate the figure, instead of fig. However, we might have to index now:
    - `nrows` = 1, `ncols` = 1: no indexing necessary
    - `nrows` > 1, `ncols` = 1: 1 index necessary: ax[...]
    - `nrows` = 1, `ncols` > 1: 1 index necessary: ax[...]
    - `nrows` > 1, `ncols` > 1: 2 indices necessary: ax[..., ...]
- Many options that we saw before in the `fig = plt.Figure()` case will now have an additional `set_`. E.g. `fig.xlabel()` is now `ax.set_xlabel()`.

Optional: Choose the amount of bins with the `bins` argument. You can also give it a nice colour with the `color` argument (str object).

Good luck!!

[<ins> Solution: click here </ins>](#solution_first_plt_plot)

<div id="section_2.3"></div>

## 2.3 Introducing: Seaborn

Potentially this already felt a bit cumbersome. Indeed, matplotlib provides you a lot of freedom, but to really get a publishable figure, we would need to write many lines of code.

Hene, sometimes it is nice to benefit from other packages that have built upon matplotlib! Let's touch upon one in particular: `Seaborn`. Import it under the name `sns`, since this is, like `plt`, a consensus in the python language. It will be handy on stackoverflow! However, you can name it any way you want. For now, run: `import seaborn as sns`.

Let's again plot a histogram. We will, however, use some knowledge we have gained on matplotlib:
1. Initiate a subplot instance: `fig, ax = plt.subplots()`
2. Assign a seaborn histogram to the ax object. Do this by assiging your `ax` object to the `ax` argument of the function: `sns.histplot(x, ax = ax)`. `x` is your data vector.

[<ins> Solution: click here </ins>](#solution_seaborn_histogram)

Noticed something? 
- Seaborn works well with pandas! The columnnames are used as labels by default.
- The figure doesn't exactly match the previous one. Try setting `bins` to 10 in the cell below. Now you can compare them :)

Let's integrate what we have learned! Can you make the plot with following descriptions:
- 2 plots above each other
- The upper is the distribution of the `whole_brain` column. Instead of a histogram, plot the Kernel Density Estimation (KDE) with `sns.kdeplot()`.
- The lower is a scatterplot between `age` (x) and `whole_brain` (y). HINT: use `sns.scatterplot()` or `sns.regplot()` (adds regression line).
- All plots should be red
- Add x and y labels

[<ins> Solution: click here </ins>](#solution_seaborn_subplots)

<div id="section_2.4"></div>

## 2.4 Manipulating an ax object in a function

Say that we have a figure with 3x3 subplots, and we want to plot all possible relations between the following features:
- `age`: amount of years from birth
- `whole_brain`: Whole brain volume 
- `Ln`: Performance on the new language being learned

It will be quite cumbersome to plot everything manually, so why not write a function? Let's go through the steps:
- Create an empty grid with `plt.subplots()`
- Write a function `add_regplot` that takes an `ax` object as argument. Within that function, manipulate the `ax` object to your preferences. Return the ax object afterwards.
- Write a nested for loop, with both loops looping over the same features stated above. HINT: the `enumerate()` function allows you to keep track of a "counter". E.g. `for index, feature in enumerate(['age', 'whole_brain', 'Ln'])` will assign the list elements to the `feature` variable, while `index` takes `0`, `1` and `2` for iteration 1, 2 and 3 respectively.
- Extract the column for each feature by `df[feature]`. Pass these, along with the subplot you want to plot it on (remember to index your `ax` object!) to the function and assign it to the same subplot.

[<ins> Solution: click here </ins>](#solution_ax_with_function)

<div id="section_2.5"></div>

## 2.5 Visualisation options for data exploration

Congratulations! With some adaptations on the former figure, you would get to what is called a **"pairplot"**. The diagonal figures are not that useful right? In a pairplot, the diagonal is the distribution of that feature, which maximizes the information inside the plot. 

Seaborn has one too: `sns.pairplot()`, and it works very well with pandas. Try it yourself! Use the function and give a pandas dataframe as argument. To reduce the size of the pairplot: Only select some columns of the dataframe.
- NOTE: There are many adaptations possible to pairplots to adapt it to your preferences (add regression lines, textboxes (bit more advanced though), change the diagonal to KDE plots, ...). For example, try changing some parameters such as `kind` and `scatter_kws`, like suggested in the following [post](https://stackoverflow.com/questions/50722972/change-the-regression-line-colour-of-seaborns-pairplot). Also, the `hue` parameter allows you to change colors based on a categorical/dichotomous variable! Try adding `hue = 'sex'` to the pairplot (CAVE: you then also need to add that column to the column selection of your dataframe).

[<ins> Solution: click here </ins>](#solution_pairplot)

A pairplot is a nice tool for data exploration. But what if we have a lot of variables? Wouldn't our pairplot become huge? Exactly. That is why you can also use a nice [heatmap](https://seaborn.pydata.org/generated/seaborn.heatmap.html) with `sns.heatmap()`. Again, this works nice with pandas, this time e.g. with the `.corr()` function applied to the dataframe. The latter function will return an n x n square matrix, with 1's on the diagonal (perfectly correlated), and the upper and lower triangle mirrored. Recognize the pairplot in this structure? Bet you did :) 

Let's make one with the desired variables!

[<ins> Solution: click here </ins>](#solution_heatmap)

***

<img src="../inputs/section_header.png">

<div id="section_3"></div>

# 3. Let's track a voxel over time!

In section 1.3, we visualised a slice of the functional imaging data. Now, let's try to do something more challenging. We will track how the BOLD value inside the voxel in a T2* image changes over the 4th dimension, time. We will also plot the location of the voxel on a slice of choice. Say we want to plot it on a sagittal slice.

We want to visualize the voxel with the following spatial characteristics:
- Medio-lateral position: 30
- Antero-posterior position: 53
- Cranio-caudal position: 20

We will give you some hints on how to get started:
- Use matplotlib `subplots`. Use 2 cols.
- On the left figure (remember to use indexing here):
    - Plot the slice. First get the slice using indexing. Take timepoint = 0. Use information from earlier in the notebook to do this.
    - Use `ax.scatter()` to plot a dot on the correct location of the voxel. For `x` and `y`, you can just use a single number. Take a nice color to contrast the rest of the image.
- On the right figure:
    - Plot the voxel value over time. Try to index the image object correctly so that a 1D vector remains, containing the BOLD values of the specific voxel over time.
    - Plot that vector using `ax.plot()`. 

[<ins> Solution: click here </ins>](#solution_voxel_over_time)

Perhaps it might be nice to see the spatial location of the tracked voxel in 3 different planes of the MRI image. Run the following code block in a next cell to reveal its location: `track_voxel(bold_image, 30, 53, 20, voxel_color='lightgreen')`.

***

<img src="../inputs/section_header.png">

<div id="section_4"></div>

# 4. Additional plots to consider
For a nice gallery of seaborn plots, click [here](https://seaborn.pydata.org/examples/index.html). Although the list is far to big to implement in 1 jupyter notebook, we will list some plots that you could consider. Perhaps you can try some of them out using the `df` of this notebook!
- `sns.jointplot()`: A scatterplot that gives you insight about the distributions of both variables? A jointplot is such an extension. The distributions are shown alongside the x and y axes
- Raincloud plot: a combination of a density plot (KDE) and a boxplot. A very nice alternative to a simple distribution with a histogram, kde plot or boxplot. This can be used via the "ptitprince" package, for which the documentation can be found [here](https://github.com/pog87/PtitPrince).

<br />

***

<div id="section_funny_note"></div>

# Funny note:

Sometimes Seaborn appears to have opinions on which colormaps you can use :) Check it out [here](https://www.reddit.com/r/Python/comments/8psk37/til_that_seaborn_refuses_to_use_the_jet_color/).

<br />

***

<div id="section_ack_ref"></div>

# Acknowledgements and References

We express our sincere gratitude to Kshipra Gurunandan, Manuel Carreiras and Pedro M. Paz-Alonso for making this [data](https://openneuro.org/datasets/ds003542) open source, and thus allowing us to make this tutorial. References and contact details can be found below:

## References for use of this data

In [49]:
print(references)
# layout.description['ReferencesAndLinks']



Publications using this dataset:

[1] Gurunandan, K., Carreiras, M., & Paz-Alonso, P.M. (2019). Functional plasticity associated with language learning in adults. NeuroImage, 201. Doi:10.1016/j.neuroimage.2019.116040
- Data: comprehension task from this dataset.
- Summary: In this work, we examined different dimensions of learning-dependent plasticity of reading and speech comprehension in intermediate and advanced adult language learners, i.e. after the initial effort of adult second language learning, do language networks continue to change with increasing proficiency?

[2] Gurunandan, K., Arnaez-Telleria, J., Carreiras, M., & Paz-Alonso, P.M. (2020). Converging evidence for differential specialization and plasticity of language systems. Journal of Neuroscience, 40(50), 9715-9724. Doi:10.1523/JNEUROSCI.0851-20.2020
- Data: comprehension and production tasks from this dataset + comprehension and production tasks from a longitudinal dataset (available at [add link]).
- Summary: In th

## Contact information of owners of the datasets

In [6]:
print('For any questions to the authors that made this dataset open-access to everyone:')
print(contact)

For any questions to the authors that made this dataset open-access to everyone:


If you have any questions, please contact:
- Kshipra Gurunandan k.gurunandan@bcbl.eu
- Pedro M. Paz-Alonso p.pazalonso@bcbl.eu



***
***

<img src="../inputs/section_header.png">

# Solutions

## Solutions section 1.2

<div id="solution_dim_anat"></div>
<ins> Solution: </ins>

In [ ]:
t1w_shape = t1w_image.shape
print(f"Shape of the 3D numpy array: {t1w_shape}")
print(f"- # Medio-lateral voxels: {t1w_shape[0]}")
print(f"- # Antero-posterior voxels: {t1w_shape[1]}")
print(f"- # Cranio-caudal voxels: {t1w_shape[2]}")

<div id="solution_show_anat_slice"></div>
<ins> Solution: </ins>

- All voxels in the Medio-lateral direction must be selected, so `:` in the fist position
- All voxels in the Antero-posterior direction must be selected, so `:` in the second position
- Only voxel number 150 must be selected in Cranio-caudal direction, so `150` in the second position

Hence, we will index the image as follows: `[:,:,150]`

In [40]:
t1w_slice = t1w_image[:, :, 150]

<IPython.core.display.Javascript object>

## Solutions section 1.3

<div id="solution_upload_bold_file"></div>
<ins> Solution: </ins>

In [47]:
bold_file = nib.load(bold_path)     # Upload the NIfTI file with nibabel
bold_image = bold_file.get_fdata()  # Get the 3D numpy array that contains grey values of the voxels

<IPython.core.display.Javascript object>

<div id="solution_dim_array_func"></div>
<ins> Solution: </ins>

In [54]:
bold_shape = bold_image.shape
print(f"Shape of the 3D numpy array: {bold_shape}")
print(f"- # Medio-lateral voxels: {bold_shape[0]}")
print(f"- # Antero-posterior voxels: {bold_shape[1]}")
print(f"- # Cranio-caudal voxels: {bold_shape[2]}")
print(f"- # Timepoints: {bold_shape[3]}")

<div id="solution_show_bold_image"></div>
<ins> Solution: </ins>

- All voxels in the Medio-lateral direction must be selected, so `:` in the fist position
- All voxels in the Medio-lateral direction must be selected, so `:` in the second position
- Only voxel number 150 must be selected in Cranio-caudal direction, so `10` in the second position
- We consider the first timepoint, so the last dimension (time) should be indexed as `0`

Hence, we will index the image as follows: `[:,:,10,0]`

In [40]:
bold_slice = bold_image[:, :, 10, 0]
fig, ax = plt.subplots()
ax.imshow(bold_slice.T, cmap="coolwarm", origin="lower")

<div id="solution_show_bold_image_question"></div>
<ins> Solution: </ins>

We see that the resolution of the fMRI is far less than the T1w image! This is why we chose axial slice 20 instead of 150 (which was the case for T1w); slice 150 doesn't exist in this fMRI image!

## Solutions section 2.2

<div id="solution_first_plt_plot"></div>
<ins> Solution: </ins>

In [20]:
import matplotlib.pyplot as plt

fig = plt.Figure()
plt.hist(df['age'])
plt.xlabel('Age [years]', fontsize = 18)
plt.ylabel('Count', fontsize = 18)
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.title('Age distribution', fontsize = 18)
plt.show()

<div id="solution_second_plt_plot"></div>
<ins> Solution: </ins>

In [19]:
fig,ax = plt.subplots(nrows = 1, ncols = 2, figsize = (10,4)) # (10,4) chosen since it looks nice
# Left figure (indexing 0)
ax[0].hist(df['age'][df['sex'] == 'M'], color = 'red')
ax[0].set_xlabel('Age [years]')
ax[0].set_ylabel('Count')
ax[0].set_title('Male')

# Right figure (indexing 0)
ax[1].hist(df['age'][df['sex'] == 'F'], color = 'green')
ax[1].set_xlabel('Age [years]')
ax[1].set_ylabel('Count')
ax[1].set_title('Female')

# Common title:
plt.suptitle('Histograms')
plt.show()

## Solutions section 2.3

<div id="solution_seaborn_histogram"></div>
<ins> Solution: </ins>

In [106]:
import seaborn as sns
fig, ax = plt.subplots()
sns.histplot(df['age'], ax = ax)
plt.show()

<IPython.core.display.Javascript object>

<div id="solution_seaborn_subplots"></div>
<ins> Solution: </ins>

In [12]:
fig, ax = plt.subplots(nrows = 2, figsize = (12,8))
sns.kdeplot(df['whole_brain'], ax = ax[0], color = 'red')
sns.regplot(x = df['age'], y = df['whole_brain'], color = 'red')
plt.show()

Nice figure eh? Only 4 lines of code!

## Solutions section 2.4

<div id="solution_ax_with_function"></div>
<ins> Solution: </ins>

In [15]:
def add_regplot(ax, x, y):
    sns.regplot(ax=ax, x=x, y=y)
    return ax

<IPython.core.display.Javascript object>

In [49]:
fig,ax = plt.subplots(nrows=3,ncols=3, figsize = (9,9))
for col_nr, feature_1 in enumerate(['age', 'whole_brain', 'Ln']):
    for row_nr, feature_2 in enumerate(['age', 'whole_brain', 'Ln']):
        ax[row_nr, col_nr] = add_regplot(ax = ax[row_nr, col_nr], x = df[feature_1], y = df[feature_2])
plt.show()

<IPython.core.display.Javascript object>

## Solutions section 2.5

<div id="solution_pairplot"></div>
<ins> Solution: </ins>

In [101]:
sns.pairplot(df[['age', 'whole_brain', 'Ln', 'sex']], diag_kind = 'kde', hue = 'sex')

<IPython.core.display.Javascript object>

<div id="solution_heatmap"></div>
<ins> Solution: </ins>

In [100]:
sns.heatmap(df.corr(), annot = True, cmap = 'coolwarm')

<IPython.core.display.Javascript object>

## Solutions section 3

<div id="solution_voxel_over_time"></div>
<ins> Solution: </ins>

In [99]:
fig, ax = plt.subplots(ncols = 2, figsize = (16,8))

# Get the vector with BOLD values over time.
voxel_BOLD_vector = bold_image[30, 53, 20, :]

# Plot the upper plot: the slice
bold_slice = bold_image[30,:,:,0]
ax[0].imshow(bold_slice.T, cmap="coolwarm", origin="lower")
ax[0].scatter(53, 20, color = 'black')

# Plot the bottom plot: the BOLD values
ax[1].plot(voxel_BOLD_vector)

plt.show()

<IPython.core.display.Javascript object>